In [2]:
# ============================================================
# 1. Setup and Library Imports
# ============================================================
!pip3 install matplotlib seaborn scikit-image tensorflow tqdm

import os
import time
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# ============================================================
# 2. Dataset Paths
# ============================================================

DATA_DIR = "sign_detection/dataset/"

# List all folders in dataset, ignoring hidden ones (like .ipynb_checkpoints)
classes = sorted([d for d in os.listdir(DATA_DIR) if not d.startswith('.')])
print("Classes found:", classes)

# ============================================================
# 3. Load and Preprocess Images
# ============================================================

IMG_SIZE = (64, 64)  # CNN input size
images, labels = [], []

for idx, cls in enumerate(classes):
    cls_path = os.path.join(DATA_DIR, cls)
    if not os.path.isdir(cls_path):
        continue
    for file in tqdm(os.listdir(cls_path), desc=f"Loading {cls}"):
        if file.startswith('.'):
            continue  # skip hidden files
        img_path = os.path.join(cls_path, file)
        try:
            img = imread(img_path)
            # Convert grayscale images to RGB
            if img.ndim == 2:
                img = np.stack([img]*3, axis=-1)
            # Resize image to CNN input size
            img_resized = resize(img, IMG_SIZE, anti_aliasing=True)
            images.append(img_resized)
            labels.append(idx)
        except Exception as e:
            print(f"Skipping {img_path}: {e}")
            continue

X = np.array(images)
y = np.array(labels)

print("Image data shape:", X.shape)
print("Unique classes:", np.unique(y))

# ============================================================
# 4. Split Data
# ============================================================

# Split data: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# Normalize pixel values to [0,1]
X_train, X_val, X_test = X_train / 255.0, X_val / 255.0, X_test / 255.0

# One-hot encode labels
num_classes = len(classes)
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print("One-hot encoded labels shape:", y_train_cat.shape)

# ============================================================
# 5. Define CNN Model
# ============================================================

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# ============================================================
# 6. Train CNN
# ============================================================

start_time = time.time()

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=50,
    batch_size=32,
    verbose=1
)

train_time = time.time() - start_time
print(f"\nTraining time: {train_time:.1f} s")

# ============================================================
# 7. Evaluate Model
# ============================================================

val_loss, val_acc = model.evaluate(X_val, y_val_cat, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Validation Accuracy: {val_acc:.3f}")
print(f"Test Accuracy: {test_acc:.3f}")

# Classification report
y_pred = np.argmax(model.predict(X_test), axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=classes))

# ============================================================
# 8. Save Model for main.ipynb
# ============================================================

model.save('cnn_model.h5')
print("CNN model saved as cnn_model.h5")

# ============================================================
# 9. Visualize Training History
# ============================================================

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


^C


ModuleNotFoundError: No module named 'matplotlib'